## Алгоритм ветвей и границ(branch and bound)

Алгоритм ветвей и границ является одним из основных алгоритмов решения целочисленных проблем линейной оптимизации.

Алгоритм заключается в последовательности следующих шагов:
    
    1. Решаем релаксационную задачу (root, node 0). Проверяем является ли решение вещественной задачи целочисленной. Если да, мы получили решение.
    2. Выбираем вещественную переменную (TODO описать алгоритмы выбора переменных) и исходную задачу делим на две - $x \geq \lceil x\rceil$ или $x \leq \lfloor x \rfloor$. Таким образом исходную задачу мы можем представить в виде бинарного дерева.
    3. Решаем задачу с добавленным ограничением. Если решение целочисленное оно становится действующим (Incumbent) оно таковым и останется до тех пор пока не будет найдено лучшее целочисленное решение. Кроме того, в случае задачи минимизации(максимизации) действующее решение является верхней(нижней) границей для оптимального решения. Это означает, что те ноды, решение в которых больше(меньше) можно не рассматривать, так как их обозрение не даст результата лучше чем действующий. (рис.1 показаны серым цветом)
    4. Продолжаем поиск до тех пор пока решение не будет найдено.  
    
Описанный метод ветвей и границ можно удобно проиллюстрировать:

![](\.imgs\BB.svg)

За основу взят пример из учебника Sarker, Newton - Optimization Modeling.

Кроме того, метод можно реализовать самостоятельно в демонстративных целях.

Опишем путь ветвления(ROOT-NODE2-NODE3-NODE6), который приводит к оптимальному решению, согласно илюстрации.
Каждый спуск по дереву добавляет к узлу новое ограничение.


In [5]:
import pyomo.environ as pyo


m = pyo.ConcreteModel('bb example')
# сразу запишем релаксационную задачу до вещественной
m.x1 = pyo.Var(within = pyo.NonNegativeReals)
m.x2 = pyo.Var(within = pyo.NonNegativeReals)
m.obj = pyo.Objective(expr = 100*m.x1 + 60*m.x2, sense=pyo.maximize)
m.machining = pyo.Constraint(expr=5*m.x1 + 2*m.x2 <= 200)
m.sanding = pyo.Constraint(expr=4*m.x1 + 3*m.x2 <= 180)
m.assembly = pyo.Constraint(expr=3*m.x1 + 4*m.x2 <= 200)
solver = pyo.SolverFactory('scip')
result = solver.solve(m, tee=False)
print('ROOT', pyo.value(m.x1,), pyo.value(m.x2), pyo.value(m.obj))
# переход в NODE 2
m.node2 = pyo.Constraint(expr=m.x1 <= 34)
result = solver.solve(m, tee=False)
print('NODE 2', pyo.value(m.x1,), pyo.value(m.x2), pyo.value(m.obj))
# 34.0 14.6 4280.0
# переход в NODE 3
m.node3 = pyo.Constraint(expr=m.x2 >= 15)
result = solver.solve(m, tee=False)
print('NODE 3', pyo.value(m.x1,), pyo.value(m.x2), pyo.value(m.obj))
# 33.75 15.9 4275.0
# переход в NODE 6
m.node4 = pyo.Constraint(expr=m.x1 <= 33)
result = solver.solve(m, tee=False)
print('NODE 6', pyo.value(m.x1,), pyo.value(m.x2), pyo.value(m.obj))
# 33.75 15.9 4275.0

ROOT 34.285714285714285 14.285714285714292 4285.714285714286
NODE 2 34.0 14.666666666666666 4280.0
NODE 3 33.75 15.0 4275.0
NODE 6 33.0 16.0 4260.0


Подобный лог можно увидеть если решать сразу целочисленную задачу решателем SCIP выключив все эвристики:

In [7]:
import pyomo.environ as pyo


m = pyo.ConcreteModel('scip')
m.x1 = pyo.Var(within = pyo.NonNegativeIntegers)
m.x2 = pyo.Var(within = pyo.NonNegativeIntegers)
m.obj = pyo.Objective(expr = 100*m.x1 + 60*m.x2, sense=pyo.maximize)
m.machining = pyo.Constraint(expr=5*m.x1 + 2*m.x2 <= 200)
m.sanding = pyo.Constraint(expr=4*m.x1 + 3*m.x2 <= 180)
m.assembly = pyo.Constraint(expr=3*m.x1 + 4*m.x2 <= 200)
solver = pyo.SolverFactory('scip')
solver.options["heuristics/vbounds/freq"] = "-1"
solver.options["heuristics/oneopt/freq"] = "-1"
solver.options["heuristics/simplerounding/freq"] = "-1"
solver.options["heuristics/randrounding/freq"] = "-1"
solver.options["heuristics/rounding/freq"] = "-1"
solver.options["heuristics/shifting/freq"] = "-1"
solver.options["heuristics/farkasdiving/freq"] = "-1"
solver.options["heuristics/alns/freq"] = "-1"


result = solver.solve(m, tee=True)

SCIP version 8.1.0 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex 6.0.4] [GitHash: 6129793871]
Copyright (c) 2002-2023 Zuse Institute Berlin (ZIB)

External libraries: 
  Soplex 6.0.4         Linear Programming Solver developed at Zuse Institute Berlin (soplex.zib.de) [GitHash: 950b1658]
  CppAD 20180000.0     Algorithmic Differentiation of C++ algorithms developed by B. Bell (github.com/coin-or/CppAD)
  ZLIB 1.2.13          General purpose compression library by J. Gailly and M. Adler (zlib.net)
  MPIR 3.0.0           Multiple Precision Integers and Rationals Library developed by W. Hart (mpir.org)
  ZIMPL 3.5.3          Zuse Institute Mathematical Programming Language developed by T. Koch (zimpl.zib.de)
  AMPL/MP 4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp)
  PaPILO 2.1.4         parallel presolve for integer and linear optimization (github.com/scipopt/papilo) [GitHash: ee0677c4]
  bliss 0.77           Computing Graph Automorphism Groups by